# Importing libraries and initializing 

In [1]:
import os
from openai import OpenAI
from pydub import AudioSegment
import re
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initialize the OpenAI client using the API key from environment variable
client = OpenAI()

In [3]:
try:
    # A simple API call to test the connection
    response = client.models.list()
    print("API connection successful!")
except Exception as e:
    print(f"Error connecting to OpenAI API: {e}")

API connection successful!


## Start loading in content and processing

In [4]:
# let's check if the file exists in the current directory
if 'raw_transcript.txt' in os.listdir():
    # If the file exists, read its contents
    with open('raw_transcript.txt', 'r', encoding='utf-8') as file:
        transcript = file.read()
    print("Transcript loaded successfully.")
else:
    print("The file 'raw_transcript.txt' was not found in the current directory.")
    print("Please ensure that the file exists and you're in the correct directory.")


Transcript loaded successfully.


In [5]:
# Split the transcript into segments for each speaker
segments = re.split(r'(SPEAKER \d+:)', transcript)[1:]  # [1:] to skip the first empty string
segments = [segments[i:i+2] for i in range(0, len(segments), 2)]

In [7]:
# segments

### Generate the actual audio and output the podcast

In [8]:
# Function to generate audio for a segment
def generate_audio(text, voice, filename):
    response = client.audio.speech.create(
        model="tts-1-hd",
        voice=voice,
        input=text
    )
    response.stream_to_file(filename)

In [9]:
# Generate audio for each segment
audio_files = []
for i, (speaker, text) in enumerate(segments):
    voice = "alloy" if "SPEAKER 1" in speaker else "nova"  # Different voices for different speakers
    filename = f"segment_{i}.mp3"
    generate_audio(text, voice, filename)
    audio_files.append(filename)

In [10]:
# Combine audio files
combined = AudioSegment.empty()
pause = AudioSegment.silent(duration=500)  # 500 ms pause

for file in audio_files:
    segment = AudioSegment.from_mp3(file)
    combined += segment + pause

# Remove the final pause
combined = combined[:-500] 

In [11]:
combined

In [12]:
# Export the final podcast
combined.export("podcast.mp3", format="mp3")

# Clean up temporary files
for file in audio_files:
    os.remove(file)

print("Podcast created successfully!")


Podcast created successfully!
